# Housekeeping

In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Gathering and Defining RSI

In [39]:
TICKER = 'SPY'
RSI_LENGTH = 14
OVERBOUGHT = 70
OVERSOLD = 30
LOOKBACK = 158

def get_data(ticker=TICKER):
    df = yf.download(ticker, period='max')
    df.columns = df.columns.get_level_values(0)
    
    return df.iloc[-LOOKBACK:, :]

def add_RSI(df, length=RSI_LENGTH):
    price_change = df['Close'].diff()
    
    # define gains/losses
    gain = price_change.where(price_change > 0, 0)
    loss = -price_change.where(price_change < 0, 0)

    # average gain vs loss
    avg_gain = gain.rolling(window=length).mean()
    avg_loss = loss.rolling(window=length).mean()

    # calculate rsi
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI'] = rsi


    plt.plot(df.index, df['RSI'])
    plt.axhline(OVERBOUGHT, color='red', linestyle='--')
    plt.axhline(OVERSOLD, color='green', linestyle='--')
    plt.xlabel('Date')
    plt.ylabel('RSI')
    plt.title('Relative Strength Index (RSI)')
    plt.grid(True)
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    plt.show()

    return df.dropna()

# Building and Testing the Strategy

In [41]:
def add_strategy(df, overbought = OVERBOUGHT, oversold = OVERSOLD):
    df['Strategy'] = 0
    
    #Where the RSI is not overbought, it is going to be -1 or 0, depending on if it is oversold or not
    df['Strategy'] = np.where(df['RSI'] > overbought, 1, 
                              np.where(df['RSI'] > oversold,-1,0))
    df['Strategy'].shift(1)
    return df

def test_strategy(df):
    df['Asset_Returns'] = (1 + df['Close'].pct_change()).cumprod() - 1
    df['Strategy_Returns'] = (1 + df['Close'].pct_change() * df['Strategy']).cumprod() -1

    plt.figure()
    plt.plot(df.index, df['Asset_Returns'], label=f'{TICKER} Buy & Hold')
    plt.plot(df.index, df['Strategy_Returns'], label='RSI Strategy')
    plt.legend()
    plt.title(f'RSI Strategy vs {TICKER}')
    plt.grid(True)
    plt.gcf().autofmt_xdate()
    plt.tight_layout()
    plt.show()

    return df

# Summarising Returns and Executing Code

In [37]:
def return_summary(df, TICKER, LOOKBACK):
    asset_return = df['Asset_Returns'].iloc[-1] * 100
    strategy_return = df['Strategy_Returns'].iloc[-1] * 100
    print(f"{TICKER} Buy & Hold Return: {asset_return:.2f}%")
    print(f"{LOOKBACK}-Day RSI Strategy Return: {strategy_return:.2f}%")
    

def main():
    df = get_data()
    df = add_RSI(df)
    df = add_strategy(df)
    df = test_strategy(df)
    return_summary(df, TICKER, LOOKBACK)
    return df

main()